<h3> 1. softmax 구현하기 </h3>

In [23]:
from common.functions import softmax
import numpy as np

## 기본적으로 데이터가 2차원으로 들어올 것이기 때문에, 교재에 있는 코드를 그대로 하면 오류가 생길 수 있음.
## 1차원일 때는 교재에 있는 코드가 잘 돌아가지만, 2차원일 때는 그렇지 않음.

def softmax(x):
    ## 2차원인 경우와 1차원인 경우로 나누어서 계산
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis = 0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        return y.T
    
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))


a = np.array([[1, 3, 5], [12, 15, 10], [13, 6, 7]])
softmax(a)

array([[1.58762400e-02, 1.17310428e-01, 8.66813332e-01],
       [4.71234165e-02, 9.46499123e-01, 6.37746092e-03],
       [9.96620823e-01, 9.08800555e-04, 2.47037604e-03]])

<h3> 3. 3층 신경망 구현하기 </h3>
문제에서 요구하는 신경망은 아래 사진과 같음

![nn](three_layer_network.png)

In [31]:
from common.functions import * ## 이전 과제에서 구현하였던 활성화 함수를 불러오기

np.random.seed(42)
def init_network(): ## 가중치와 편향 저장
    network = {}
    network['W1'] = np.random.randn(2, 2) ## 입력층 to 첫번째 은닉층 가중치
    network['b1'] = np.random.randn(2) ## 첫번째 은닉층(h1, h2)에 들어가는 편향 2개
    network['W2'] = np.random.randn(2, 3) ## 첫번째 은닉층 to 두번째 은닉층 가중치
    network['b2'] = np.random.randn(3) ## 두번째 은닉층(s1, s2, s3)에 들어가는 가중치
    network['W3'] = np.random.randn(3, 3) ## 두번째 은닉층 to 출력층 가중치
    network['b3'] = np.random.randn(3) ## 출력층에 들어가는 가중치
    
    return network

def forward(network, x):
    ## 입력층 to 출력층으로 가는 연산 과정
    W1, W2, W3 = network['W1'], network['W2'], network['W3'] ## 신경망 계산에 쓰이는 가중치 불러오기
    b1, b2, b3 = network['b1'], network['b2'], network['b3'] ## 신경망 계산에 쓰이는 편향 불러오기
    
    
    ## 입력층부터 출력층까지 연산 과정
    a1 = np.dot(x, W1) + b1 ## 입력층과 첫번째 은닉층 사이의 내적
    z1 = sigmoid(a1) ## 첫번째 은닉층에 활성화함수 적용
    a2 = np.dot(z1, W2) + b2 ## 첫번째 은닉층과 두번째 은닉층 사이의 내적
    z2 = sigmoid(a2) ## 두번째 은닉층에 활성화함수 적용
    a3 = np.dot(z2, W3) + b3 ## 두번째 은닉층과 출력층 사이의 내적
    y = identity_function(a3) ## 마지막 출력층에 활성화함수 적용
    
    return y

network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

[-1.17182067 -0.94161836 -0.79266835]


<h3> 4. MNIST 데이터를 활용하여 신경망의 추론 과정을 구현하기 </h3>
<h5> (1) 코드 import </h5>

In [36]:
import os, sys
sys.path.append('********************')
from mnist import load_mnist

## 데이터 불러오기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


<h5> (2) MNIST 데이터를 이용하여 추론 과정 구현 </h5>

In [44]:
import pickle 


def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)
    
    return x_test, t_test

def init_network():
    with open('sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
        
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

## Custom Model 만들기
def init_network2():
    network = {}
    network['W1'] = np.random.randn(784, 50)
    network['b1'] = np.random.randn(50)
    network['W2'] = np.random.randn(50, 100)
    network['b2'] = np.random.randn(100)
    network['W3'] = np.random.randn(100, 10)
    network['b3'] = np.random.randn(10)
    
    return network

     
x, t = get_data()
network = init_network()
network2 = init_network2()

accuracy_cnt = 0
accuracy_cnt2 = 0

## Custom Model과 사전학습된 모델간의 정확도 차이 계산하기
for i in range(len(x)):
    y = predict(network, x[i]) ## 사전학습된 sample_weight.pkl에 의한 예측 결과
    y2 = predict(network2, x[i]) ## 사전학습되지 않은 custom model에 의한 예측 결과
    
    p = np.argmax(y) ## sample_weight의 예측 최종 결과
    p2 = np.argmax(y2) ## custom model에 의한 예측 최종 결과
    
    if p == t[i]:
        accuracy_cnt += 1
        
    if p2 == t[i]:
        accuracy_cnt2 += 1
        
print('학습된 network의 Accuracy: ' + str(float(accuracy_cnt) / len(x)))
print('학습되지 않은 network의 Accuracy: ' + str(float(accuracy_cnt2) / len(x)))

## 차이나는 이유 -> sample_weight.pkl은 학습된 weight이고, network2에 구현한 것은 학습되지 않은 weight임.

학습된 network의 Accuracy: 0.9207
학습되지 않은 network의 Accuracy: 0.1131


<h3> (3) 배치 처리 </h3>

In [48]:
batch_size = 100 ## 배치 처리 개수

accuracy_cnt = 0

## 한번에 100개 처리하기
for i in range(0, len(x), batch_size):
    x_test, t_test = x[i:i+batch_size], t[i:i+batch_size]
    y_pred = np.argmax(predict(network, x_test), axis = 1)
    accuracy_cnt += np.sum(y_pred == t_test)
    
print('배치 처리 후 학습된 network의 Accuracy: ' + str(float(accuracy_cnt) / len(x)))

## 2.9초 vs 0.1초 차이
    

배치 처리 후 학습된 network의 Accuracy: 0.9207
